<h1><center>ENGG1200 Data Analysis</center></h1>

<img src="analytics.jpg" width="500" height="100" align="center"/>

In [1]:
#import libraries
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
import nltk as nl
nl.download('punkt',quiet=True)
nl.download('stopwords',quiet=True)
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import collections
from nltk.text import Text
import re 
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from operator import itemgetter
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from ipywidgets import Button, HBox, VBox, Layout, Box
from IPython.display import display
from IPython.display import clear_output
import pyLDAvis
import pyLDAvis.gensim
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis.sklearn
from pivottablejs import pivot_ui
np.warnings.filterwarnings('ignore')

words = ['Statistics','Visualization', 'WordClouds','Topic Modelling','Sentiment Analysis','Reflections','More Visualization']
items = [Button(description=w,button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width='175px')) for w in words]
display(HBox([item for item in items]))
np.warnings.filterwarnings('ignore')
def stats(b):
    clear_output()
    display(HBox([item for item in items]))
    #import dataset 
    file = r'engg.xls'
    df = pd.read_excel(file)
    df_cleaned=df.dropna()
    df_cleaned = df_cleaned[df_cleaned['Answer/Reflection'] != '-No Answer-']
    df_cleaned = df_cleaned.reset_index(drop=True)
    #info about original and cleaned data
    print("\nThere are {} observations and {} features in original dataset and {} observations and {} features in cleaned dataset. \n".format(df.shape[0],df.shape[1],df_cleaned.shape[0],df_cleaned.shape[1])) 
    print("\nA peek into originl data\n")
    display(df.head())
    print("\n\nA peek into cleaned data\n")
    df_cleaned=df.dropna()
    df_cleaned = df_cleaned[df_cleaned['Answer/Reflection'] != '-No Answer-']
    df_cleaned = df_cleaned.reset_index(drop=True)
    display(df_cleaned.head())
    print("\n\nGrouped by Gender (after cleaning)\n")
    #group by gender
    gender = df_cleaned.groupby("Gender")
    display(gender.describe().head())
    #group by international students
    print("\n\nGrouped by International-Domestic students (after cleaning)\n")
    international = df_cleaned.groupby("International")
    display(international.describe().head())
items[0].on_click(stats)

def visual(b):
    clear_output()
    display(HBox([item for item in items]))
    #import dataset 
    file = r'engg.xls'
    df = pd.read_excel(file)
    #cleaning data-removing NaN values
    df_cleaned=df.dropna()
    df_cleaned = df_cleaned[df_cleaned['Answer/Reflection'] != '-No Answer-']
    df_cleaned = df_cleaned.reset_index(drop=True)
    #extracting reviews 
    all_reviews=df_cleaned['Answer/Reflection']
    print("\n\n")
    #entire cohort group by gender
    entire_gender = df.groupby("Gender")
    #plot graph of entire gender distribution
    gender_labels = 'Male', 'Female'
    plt.figure(figsize=(10,10))
    ax = plt.subplot(221)
    ax.set_aspect(1)

    entire_gender.size().sort_values(ascending=False).plot.pie(labels = gender_labels, autopct='%1.1f%%')
    plt.ylabel('')
    plt.title('Gender Distribution in ENGG1200')
    plt.grid(True)
    
    #entire cohort group by international/domestic
    entire_international = df.groupby("International")
    #plot graph of int/dom distribution
    plt.subplot(222)
    international_domestic_labels = 'Domestic', 'International'
    entire_international.size().sort_values(ascending=False).plot.pie(labels = international_domestic_labels, autopct='%1.1f%%')
    plt.ylabel('')
    plt.title('International-Domestic Students Distribution in ENGG1200')
    plt.tight_layout()

    #group by gender
    gender = df_cleaned.groupby("Gender")
    #plot graph of gender vs reviews
    plt.figure(figsize=(10,10))
    plt.subplot(223)
    gender.size().sort_values(ascending=False).plot.bar()
    plt.xticks(rotation=50)
    plt.xlabel('')
    plt.xticks(np.arange(2), ('Male', 'Female'))
    plt.ylabel("Number of reflections")
    #might need to come up with a better graph title
    plt.title('Valid Reflections Grouped By Gender')
    
    
    #valid reflections group by international students
    international = df_cleaned.groupby("International")
    #plot graph of international vs reviews
    plt.subplot(224)
    international.size().sort_values(ascending=False).plot.bar()
    plt.xticks(rotation=50)
    plt.xticks(np.arange(2), ('Domestic', 'International'))
    plt.ylabel("Number of reflections")
    plt.xlabel('')
    #might need to come up with a better graph title
    plt.title('Valid Reflections Grouped By International-Domestic Students')
    plt.subplots_adjust(bottom=0.1, right=1.5, top=0.9)
    plt.show()
    
    #count
    male_count = df.loc[df['Gender'] == 'M'].shape[0]
    female_count = df.loc[df['Gender'] == 'F'].shape[0]
    domestic_count = df.loc[df['International'] == 'N'].shape[0]
    international_count = df.loc[df['International'] == 'Y'].shape[0]
    male_valid_count = df_cleaned.loc[df_cleaned['Gender'] == 'M'].shape[0]
    female_valid_count = df_cleaned.loc[df_cleaned['Gender'] == 'F'].shape[0]
    domestic_valid_count = df_cleaned.loc[df_cleaned['International'] == 'N'].shape[0]
    international_valid_count = df_cleaned.loc[df_cleaned['International'] == 'Y'].shape[0]

    #text
    print("\nThere are {} students in total.\nMale: {} Female: {}.\nDomestic: {} International: {}.\n".format(df.shape[0],male_count, female_count, domestic_count, international_count))
    print("\nThere are {} valid reflections (reflections in cleaned data) in total.\nMale: {} Female: {}.\nDomestic: {} International: {}.\n".format(df_cleaned.shape[0],male_valid_count, female_valid_count, domestic_valid_count, international_valid_count))



items[1].on_click(visual) 

def wc(b):
    clear_output()
    display(HBox([item for item in items]))
    print("\nPlease wait for the output to be generated.This may take a while.\n\n")
    #import dataset 
    file = r'engg.xls'
    df = pd.read_excel(file)
    #cleaning data-removing NaN values
    df_cleaned=df.dropna()
    df_cleaned = df_cleaned[df_cleaned['Answer/Reflection'] != '-No Answer-']
    df_cleaned = df_cleaned.reset_index(drop=True)
    #extracting reviews 
    all_reviews=df_cleaned['Answer/Reflection']
    #tokenization,removing stopwords, punctuation and stemming
    all_ans=""
    for review in all_reviews:
        all_ans=all_ans+review+"\n"
    all_ans= all_ans.replace("'", "")
    tokens=word_tokenize(all_ans)
    tokens=[w.lower() for w in tokens]
    text = nl.Text(tokens)
    token_words=[word for word in tokens if word.isalpha()]
    stopword=stopwords.words('english')
    stopword.append('dont')
    stopword.append('didnt')
    stopword.append('doesnt')
    stopword.append('cant')
    stopword.append('couldnt')
    stopword.append('couldve')
    stopword.append('im')
    stopword.append('ive')
    stopword.append('isnt')
    stopword.append('theres')
    stopword.append('wasnt')
    stopword.append('wouldnt')
    stopword.append('a')
    stopword.append('also')
    token_words=[w for w in token_words if not w in stopword]
    porter =PorterStemmer()
    token_stemmed=[porter.stem(w) for w in token_words]
    clear_output()
    display(HBox([item for item in items]))
    #creating worlcloud
    cloudstring=(" ").join(token_words)
    wordcloud = WordCloud(max_font_size=50,max_words=100, background_color="black").generate(cloudstring)
    plt.figure(figsize=(20,20))
    ax = plt.subplot(221)
    # plot wordcloud in matplotlib
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("WordCloud generated from reflections")
    plt.grid(True)
    #plotting bi-gram cloud
    # setup and score the bigrams using the raw frequency.
    finder = BigramCollocationFinder.from_words(token_words)
    bigram_measures = BigramAssocMeasures()
    scored = finder.score_ngrams(bigram_measures.raw_freq)
    scoredList = sorted(scored, key=itemgetter(1), reverse=True)
    word_dict = {} 
    listLen = len(scoredList)
    for i in range(listLen):
        word_dict['_'.join(scoredList[i][0])] = scoredList[i][1]

    wordCloud = WordCloud(max_font_size=50, max_words=100, background_color="black")
    plt.subplot(222) 
    wordCloud.generate_from_frequencies(word_dict)

    plt.title('Most frequently occurring bigrams connected with an underscore_')
    plt.imshow(wordCloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    
    #plotting frequency distribution
    plt.figure(figsize=(25,5))
    ax = plt.subplot(121)
    freqdist = nl.FreqDist(token_words)
    plt.subplot(121) 
    plt.title("Frequency Distribution of token words")
    freqdist.plot(50)
    
    #dispertion plot
    plt.figure(figsize=(25,5))
    ax = plt.subplot(122) 
    text.dispersion_plot([str(w) for w, f in freqdist.most_common(20)])

    
items[2].on_click(wc)    


def tm(b):
    clear_output()
    display(HBox([item for item in items]))
    print("\nPlease wait for the output to be generated.This may take a while.\n\n")
    #import dataset 
    file = r'engg.xls'
    df = pd.read_excel(file)
    #cleaning data-removing NaN values
    df_cleaned=df.dropna()
    df_cleaned = df_cleaned[df_cleaned['Answer/Reflection'] != '-No Answer-']
    df_cleaned = df_cleaned.reset_index(drop=True)
    NUM_TOPICS = 10
    data=df_cleaned['Answer/Reflection'] 
    vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                                 stop_words='english', lowercase=True, 
                                 token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
    data_vectorized = vectorizer.fit_transform(data)

    # Build a Latent Dirichlet Allocation Model
    lda_model = LatentDirichletAllocation(n_topics=NUM_TOPICS, max_iter=10, learning_method='online')
    lda_Z = lda_model.fit_transform(data_vectorized)

    clear_output()
    display(HBox([item for item in items]))
    # Visualize the topics
    pyLDAvis.enable_notebook()
    panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
    display(panel)

items[3].on_click(tm)

def sa(b):
    clear_output()
    display(HBox([item for item in items]))
    print("\nPlease wait for the output to be generated.This may take a while.\n\n")
    #import dataset 
    file = r'engg.xls'
    df = pd.read_excel(file)
    #cleaning data-removing NaN values
    df_cleaned=df.dropna()
    df_cleaned = df_cleaned[df_cleaned['Answer/Reflection'] != '-No Answer-']
    df_cleaned = df_cleaned.reset_index(drop=True)
    all_reviews=df_cleaned['Answer/Reflection']
    #sentiment analysis
    analyser = SentimentIntensityAnalyzer()
    international_students = df_cleaned.loc[df_cleaned['International'] == 'Y']
    international_reflections = international_students['Answer/Reflection']
    domestic_students = df_cleaned.loc[df_cleaned['International'] == 'N']
    domestic_reflections = domestic_students['Answer/Reflection']
    pos_entire=[]
    neg_entire=[]
    neutral_entire=[]

    pos_int=[]
    neg_int=[]
    neutral_int=[]

    pos_dom=[]
    neg_dom=[]
    neutral_dom=[]
    for review in all_reviews:
        scores = analyser.polarity_scores(review)
        if scores['compound']<=-0.5:
            neg_entire.append(review)
        if scores['compound']>=0.5:
            pos_entire.append(review)
        if scores['compound']>-0.5 and scores['compound']<0.5:
            neutral_entire.append(review)
    clear_output()
    display(HBox([item for item in items]))        
    type_length=[len(pos_entire),len(neutral_entire),len(neg_entire)]
    sent_type=['positive','neutral','negative']
    plt.pie(type_length, labels=sent_type, startangle=90, autopct='%.1f%%')
    plt.title('Sentiment distribution for entire cohort')
    plt.show()


    for review in international_reflections:
        scores = analyser.polarity_scores(review)
        if scores['compound']<=-0.5:
            neg_int.append(review)
        if scores['compound']>=0.5:
            pos_int.append(review)
        if scores['compound']>-0.5 and scores['compound']<0.5:
            neutral_int.append(review)
            
            
    type_length=[len(pos_int),len(neutral_int),len(neg_int)]
    sent_type=['positive','neutral','negative']
    plt.pie(type_length, labels=sent_type, startangle=90, autopct='%.1f%%')
    plt.title('Sentiment distribution for international students cohort')
    plt.show()

    for review in domestic_reflections:
        scores = analyser.polarity_scores(review)
        if scores['compound']<=-0.5:
            neg_dom.append(review)
        if scores['compound']>=0.5:
            pos_dom.append(review)
        if scores['compound']>-0.5 and scores['compound']<0.5:
            neutral_dom.append(review)
    
    type_length=[len(pos_dom),len(neutral_dom),len(neg_dom)]
    sent_type=['positive','neutral','negative']
    plt.pie(type_length, labels=sent_type, startangle=90, autopct='%.1f%%')
    plt.title('Sentiment distribution for domestic students cohort')
    plt.show()

    bars1 = [len(pos_entire), len(pos_dom), len(pos_int)]
    bars2 = [len(neutral_entire), len(neutral_dom), len(neutral_int)]
    bars3 = [len(neg_entire), len(neg_dom), len(neg_int)]
    sent_type=['positive','neutral','negative']

    # set width of bar
    barWidth = 0.25
    # Set position of bar on X axis
    r1 = np.arange(len(bars1))
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]

    # Make the plot
    plt.figure(figsize=(15,10))
    plt.bar(r1, bars1, width=barWidth, edgecolor='white', label='Positive')
    plt.bar(r2, bars2, width=barWidth, edgecolor='white', label='Neutral')
    plt.bar(r3, bars3, width=barWidth, edgecolor='white', label='Negative')

    # Add xticks on the middle of the group bars
    plt.xlabel('Group', fontweight='bold')
    plt.xticks([r + barWidth for r in range(len(bars1))], ['Entire Cohort', 'Domestic Students', 'International Students'])


    # Create legend & Show graphic
    plt.legend()
    plt.title('Sentiment Distributions')
    plt.show()
    
items[4].on_click(sa)

def ref(b):
    clear_output()
    display(HBox([item for item in items]))
    print("\nPlease wait for the output to be generated.This may take a while.\n\n")
    #import dataset 
    file = r'engg.xls'
    df = pd.read_excel(file)
    #cleaning data-removing NaN values
    df_cleaned=df.dropna()
    df_cleaned = df_cleaned[df_cleaned['Answer/Reflection'] != '-No Answer-']
    df_cleaned = df_cleaned.reset_index(drop=True)
    all_reviews=df_cleaned['Answer/Reflection']
    pos_entire=[]
    neg_entire=[]
    neutral_entire=[]
    analyser = SentimentIntensityAnalyzer()
    for review in all_reviews:
        scores = analyser.polarity_scores(review)
        if scores['compound']<=-0.5:
            neg_entire.append(review)
        if scores['compound']>=0.5:
            pos_entire.append(review)
        if scores['compound']>-0.5 and scores['compound']<0.5:
            neutral_entire.append(review)
    #sentiment analysis
    clear_output()
    display(HBox([item for item in items]))
    bpos = widgets.Button(description="Next positive reflection",button_style='success',layout=Layout(width='175px'))
    bneg = widgets.Button(description="Next negative reflection",button_style='danger',layout=Layout(width='175px'))
    bneu = widgets.Button(description="Next neutral reflection",button_style='info',layout=Layout(width='175px'))
    display(HBox([bpos,bneu,bneg]))
    
    
    def posref(b):
        clear_output()
        display(HBox([item for item in items]))
        display(HBox([bpos,bneu,bneg]))
        print("\nPositive reflection\n")
        display(pos_entire[0])
        item_rem=pos_entire.pop(0)
        pos_entire.append(item_rem)
        
        
        
    bpos.on_click(posref)
    
    def neuref(b):
        clear_output()
        display(HBox([item for item in items]))
        display(HBox([bpos,bneu,bneg]))
        print("\nNeutral reflection\n")
        display(neutral_entire[0])
        item_rem=neutral_entire.pop(0)
        neutral_entire.append(item_rem)
        
        
        
    bneu.on_click(neuref)
    
    def negref(b):
        clear_output()
        display(HBox([item for item in items]))
        display(HBox([bpos,bneu,bneg]))
        print("\nNegative reflection\n")
        display(neg_entire[0])
        item_rem=neg_entire.pop(0)
        neg_entire.append(item_rem)
        
        
        
    bneg.on_click(negref)
    
items[5].on_click(ref)  

def mv(b):
    clear_output()
    display(HBox([item for item in items]))
    #import dataset 
    file = r'engg.xls'
    df = pd.read_excel(file)
    #cleaning data-removing NaN values
    df_cleaned=df.dropna()
    df_cleaned = df_cleaned[df_cleaned['Answer/Reflection'] != '-No Answer-']
    df_cleaned = df_cleaned.reset_index(drop=True)
    display(pivot_ui(df_cleaned))
    
items[6].on_click(mv)
np.warnings.filterwarnings('ignore')
    
    

C:\Users\shiva\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
